#Cell-Cell Interactions - Notebook 1/2
##Notebook purpose: Create input files for CellPhoneDB



###Setup Notebook for data extraction

In [ ]:
#mount access to google drive to access relevant data for input and output files
from google.colab import drive
drive.mount('/mntDrive')

Mounted at /mntDrive


To Do: Click "RESTART RUNTIME" button at end of code output after running pip install cell below!

In [ ]:
#Need to install scanpy for data import and extraction process, this requires restarting runtime 
!pip install scanpy

In [ ]:
#change to relevant directory in mounted drive - you can now assume you are working from this directory
%cd /mntDrive/MyDrive/Capstone/CellphoneDB_Files

/mntDrive/.shortcut-targets-by-id/1z3q46vaebOyzwaeo5O--GgQmX-22nmRh/Capstone/CellphoneDB_Files


In [ ]:
#import packages needed for data extraction
import pandas as pd
import scanpy as sc
import os 

##Import Data and Transform

TO DO: fill filepath variable in with your desired input file, .h5ad format expected. 

In [ ]:
adata_filepath = '</folder/folder/data_analysis_output_file.h5ad>' 
adata = sc.read(adata_filepath)
#visual inspection of file format -- may be needed to specify cell type label layer
adata

Create Counts Data File: 

columns = cells, rows = genes (ensemble IDs) 

In [ ]:
#extracts counts layer of anndata file
df_expr_matrix = adata.to_df(layer="counts")
#transpose dataframe to fit column & row requirements
df_expr_matrix = df_expr_matrix.T
# # Set cell ids as column headers
df_expr_matrix.columns = adata.obs.index
# # Set rows/genes to Ensemble ID
df_expr_matrix.set_index(adata.var.gene_ids, inplace=True) 
# # Visual inspection for correct format
df_expr_matrix.head()

Prepare Meta Data File:

2 columns: cell & cell type label

To Do: find correct layer in 'obs' that houses cell to cell type mapping. Leverage adata file organization printed above.

In [ ]:
adata.obs['<layer that specifies cell type label>']

In [ ]:
## generate meta file
df_meta = pd.DataFrame(data={'Cell': list(adata.obs['<layer that specifies cell type label>'].index), 
                             'cell_type': list(adata.obs['<layer that specifies cell type label>'])})

df_meta.set_index('Cell',inplace=True)

#check for missing values
print('Missing Values Count:', df_meta.isna().sum().sum())

#visuale inspection of table
df_meta.head()

If Missing Values Count >0, follow next steps, otherwise skip to "Save Extracted Data" section

To Do: Ensure shape matches between meta table # cells & counts table # cells 

In [ ]:
#create list of cells missing celltype label
drop_cells = list(df_meta[df_meta['cell_type'].isna()].index)

#drop cells missing celltype label from meta dataframe
df_meta = df_meta.dropna()
print('Shape of Meta Table:', df_meta.shape)

#update counts table to remove empty cell type
df_counts = df_expr_matrix.drop(columns=drop_cells)
print('Shape of Counts Table:', df_counts.shape)

##Save Extracted Data to Google Drive

To Do:
Specify output path! These files will be the input for CellPhoneDB. 

Can take up to 1-1.5 hours to save for large files.

In [ ]:
output_filepath = '</base_folder/output_folder/>'

In [ ]:
#saves as .txt file, tab delimited - can take up to an hour to run for large input data
df_meta.to_csv(os.join.path(output_filepath, 'meta_data.txt'), sep='\t')
df_counts.to_csv(os.join.path(output_filepath, 'counts_data.txt'), sep='\t') 